## **Image Correspondence**
This notebook will be used to build the pipeline and experiment with image correspondence.

In [ ]:
import numpy as np
import math
import cv2
import matplotlib.pyplot as plt
from scipy.stats import norm, truncnorm, t, cauchy
from scipy.special import kl_div
from scipy.signal import find_peaks

import os

from reconstruction_deep_network.preprocessing.feature_points import FeaturePointDetector, VisualizeCorners
from reconstruction_deep_network.preprocessing.optical_flow import FeatureMatching, MatchVisualizer
from reconstruction_deep_network.data_loader.matterport import MatterPortData

import reconstruction_deep_network

In [ ]:
module_dir = reconstruction_deep_network.__path__[0]
results_dir = os.path.join(module_dir, "results")

In [ ]:
## Initialize dataset
scan_hash = "17DRP5sb8fy"
panorama_id = "0f37bd0737e349de9d536263a4bdd60d"
data_loader = MatterPortData(scan_hash)

color_image_1 = data_loader.load_color_image(panorama_id, 1, 4)
color_image_2 = data_loader.load_color_image(panorama_id, 2, 4)

In [ ]:
## detect feature points
feature_point_det = FeaturePointDetector(2000)
kp_1, des_1 = feature_point_det.sift_corners(color_image_1)
kp_2, des_2 = feature_point_det.sift_corners(color_image_2)


In [ ]:
## visualize features
feature_vis = VisualizeCorners()
kp_image_1 = feature_vis.visualize_keypoints(color_image_1, kp_1)
kp_image_2 = feature_vis.visualize_keypoints(color_image_2, kp_2)


fig, axs = plt.subplots(1, 2, figsize=(10, 10))
axs[0].imshow(kp_image_1)
axs[1].imshow(kp_image_2)
plt.show()

In [ ]:
## match features
ratios = np.linspace(0.3, 0.7, 5)
feature_match = FeatureMatching()
feature_match_dict = {}
for ratio in ratios:
    matches, mask = feature_match.flann_matching(des_1, des_2, "sift", ratio)
    feature_match_dict[f"{ratio}"] = (matches, mask)

## visualize feature matches
match_visualizer = MatchVisualizer()

for ratio in feature_match_dict:
    matches, mask = feature_match_dict[ratio]
    img = match_visualizer.visualize_matches(color_image_1, color_image_2, kp_1, kp_2, matches, mask)

    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.title(f"Ratio: {ratio}")
    plt.show()

## **Feature Matches Between Distinct Images**
This is to investigate if matches are still provided for distinct images, using a ratio threshold of 0.5 from the previous result

In [ ]:
test_image = data_loader.load_color_image(panorama_id, 1, 0)
test_kp, test_des = feature_point_det.sift_corners(test_image)

test_match, test_mask = feature_match.flann_matching(des_1, test_des, "sift", 0.5)
test_img = match_visualizer.visualize_matches(color_image_1, test_image, kp_1, test_kp, test_match, test_mask)

plt.figure(figsize=(10, 10))
plt.imshow(test_img)
plt.show()

In [ ]:
test_match, test_mask = feature_match.flann_matching(des_2, test_des, "sift", 0.5)
test_img = match_visualizer.visualize_matches(color_image_2, test_image, kp_2, test_kp, test_match, test_mask)

plt.figure(figsize=(10, 10))
plt.imshow(test_img)
plt.show()

Conclusion: If two images are distinct then with the ratio test, we can eliminate any false positives. We can track the match count from the pair of images and look for similarity.

## **Feature Points in Empty Images**
This will be used to inspect feature points in plane surface images. Hypothesis: there should not be many feature points in empty surface images. Example: walls, ceilings etc.

In [ ]:
empty_image = data_loader.load_color_image(panorama_id, 1, 1)
empty_kp, empty_des = feature_point_det.sift_corners(empty_image)

enpty_kp_img = feature_vis.visualize_keypoints(empty_image, empty_kp)

plt.figure(figsize=(8, 8))
plt.imshow(enpty_kp_img)
plt.show()

### **Utility Functions**

In [ ]:
def color_based_segmentation(img: np.ndarray):
    lab_image = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

    lower_range = np.array([150, 128, 128])
    upper_range = np.array([200, 128, 128])

    mask = cv2.inRange(lab_image, lower_range, upper_range)

    result = cv2.bitwise_and(img, img, mask=mask)
    return result

def edge_detection(img: np.ndarray):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray_img, threshold1=100, threshold2=200)
    return edges

def blob_detection(img: np.ndarray):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    log_image = cv2.Laplacian(gray_img, cv2.CV_64F)
    log_image = cv2.convertScaleAbs(log_image)

    _, binary_image = cv2.threshold(log_image, 30, 255, cv2.THRESH_BINARY)
    return binary_image

def histogram_analysis(img: np.ndarray):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    histogram = cv2.calcHist([gray_img], [0], None, [256], [0, 256])
    histogram = histogram.squeeze()
    normalized_hist = ((histogram - np.min(histogram)) / (np.max(histogram) - np.min(histogram)))
    variance = np.var(normalized_hist)
    return normalized_hist, variance

def model_normal_dist(histogram: np.ndarray):
    x_mean = np.sum(np.arange(len(histogram)) * histogram) / np.sum(histogram)
    std_dev = np.std(histogram)
    x = np.linspace(0, 256, len(histogram))
    pdf_normal = norm.pdf(x, loc=x_mean, scale=std_dev)
    pdf_normal = np.clip(pdf_normal, 0, 256)
    pdf_normal /= np.sum(pdf_normal)
    return pdf_normal

def get_truncated_normal(histogram: np.ndarray):
    mean = np.sum(np.arange(len(histogram)) * histogram) / np.sum(histogram)
    low, upp = 0, 256
    sd = 1
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

def model_cauchy_distribution(histogram: np.ndarray):
    x_mean = np.sum(np.arange(len(histogram)) * histogram) / np.sum(histogram)
    scale_parameter = 20  # You can adjust this value
    x = np.linspace(0, 256, len(histogram))
    pdf_cauchy = cauchy.pdf(x, loc=x_mean, scale=scale_parameter)    

    # Calculate the PDF of the Student's t-distribution
    pdf_cauchy = np.clip(pdf_cauchy, 0, 256)
    pdf_cauchy /= np.sum(pdf_cauchy)
    return pdf_cauchy

def detect_and_draw_lines(raw_image, threshold1=50, threshold2=200, hough_threshold=100):

    image = raw_image.copy()
        
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Canny edge detection
    edges = cv2.Canny(gray, None, threshold1, threshold2)
    
    # Use the Hough Line Transform to detect lines
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 150, None, 0, 0)
    
    line_centers = []
    line_angles_u = []
    line_angles_v = []

    if lines is not None:
        # Draw detected lines on the original image
        for i in range(len(lines)):
            # rho, theta = line[0]
            # a = np.cos(theta)
            # b = np.sin(theta)
            # x0 = a * rho
            # y0 = b * rho
            # x1 = int(x0 + 1000 * (-b))
            # y1 = int(y0 + 1000 * (a))
            # x2 = int(x0 - 1000 * (-b))
            # y2 = int(y0 - 1000 * (a))
            l = lines[i][0]
            x_center = (l[0] + l[2] )/ 2
            y_center = (l[1] + l[3]) / 2
            u = abs(l[2] - l[0])
            v = abs(l[3] - l[1])

            line_centers.append([x_center, y_center])
            line_angles_u.append(u)
            line_angles_v.append(v)
            image = cv2.line(image, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3)
            
    
    return image, np.array(line_centers), np.array(line_angles_u), np.array(line_angles_v)


In [ ]:
## histogram analysis
hist, variance = histogram_analysis(empty_image)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Plot the image on the left subplot
ax2.set_title('Image')
ax2.imshow(empty_image)
ax2.axis('off')

# Plot the histogram on the right subplot
ax1.set_title(f'Histogram, variance: {variance}')
ax1.set_xlabel('Pixel Intensity')
ax1.set_ylabel('Frequency')
ax1.bar(np.arange(256), hist, color='gray')
ax1.set_xlim([0, 256])

plt.subplots_adjust(wspace=0.02)
plt.show()

In [ ]:
## histogram analysis
hist_texture, variance_texture = histogram_analysis(color_image_1)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Plot the image on the left subplot
ax2.set_title('Image')
ax2.imshow(color_image_1)
ax2.axis('off')

# Plot the histogram on the right subplot
ax1.set_title(f'Histogram, variance: {variance_texture}')
ax1.set_xlabel('Pixel Intensity')
ax1.set_ylabel('Frequency')
ax1.bar(np.arange(256), hist_texture, color='gray')
ax1.set_xlim([0, 256])

plt.subplots_adjust(wspace=0.02)
plt.show()

Conclusion: the variance of the histograms might not be enough to differentiate between the textureless image and an image with lot of textures.

In [ ]:
# model normal distribution
hist_norm_textureless = model_cauchy_distribution(hist)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Plot the histogram on the right subplot
ax1.set_title(f'Histogram: Normal')
ax1.set_xlabel('Pixel Intensity')
ax1.set_ylabel('Frequency')
ax1.bar(np.arange(256), hist_norm_textureless, color='gray')
ax1.set_xlim([0, 256])

# Plot the histogram on the right subplot
ax2.set_title(f'Histogram: Textureless')
ax2.set_xlabel('Pixel Intensity')
ax2.set_ylabel('Frequency')
ax2.bar(np.arange(256), hist, color='gray')
ax2.set_xlim([0, 256])


In [ ]:
epsilon = 1e-5
divergence = np.sum(kl_div(hist_norm_textureless + epsilon, hist + epsilon))
divergence

In [ ]:
hist_norm_texture = model_cauchy_distribution(hist_texture)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Plot the histogram on the right subplot
ax1.set_title(f'Histogram: Normal')
ax1.set_xlabel('Pixel Intensity')
ax1.set_ylabel('Frequency')
ax1.bar(np.arange(256), hist_norm_texture, color='gray')
ax1.set_xlim([0, 256])

# Plot the histogram on the right subplot
ax2.set_title(f'Histogram: Texture')
ax2.set_xlabel('Pixel Intensity')
ax2.set_ylabel('Frequency')
ax2.bar(np.arange(256), hist_texture, color='gray')
ax2.set_xlim([0, 256])


In [ ]:
divergence = np.sum(kl_div(hist_norm_texture + epsilon, hist_texture + epsilon))
divergence

In [ ]:
color_image_3 = data_loader.load_color_image(panorama_id, 0, 1)
## histogram analysis
hist_3, variance_3 = histogram_analysis(color_image_3)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Plot the image on the left subplot
ax2.set_title('Image')
ax2.imshow(color_image_3)
ax2.axis('off')

# Plot the histogram on the right subplot
ax1.set_title(f'Histogram, variance: {variance_3}')
ax1.set_xlabel('Pixel Intensity')
ax1.set_ylabel('Frequency')
ax1.bar(np.arange(256), hist_3, color='gray')
ax1.set_xlim([0, 256])

plt.subplots_adjust(wspace=0.02)
plt.show()

Conclusion: Identifying the modes of the distribution should be the way ahead. If there is at least one more mode, we can surely say that the image might have different textures.

In [ ]:
peaks_1 = find_peaks(hist, height=0.3, width=2)
peaks_2 = find_peaks(hist_texture, height=0.3, width=2)
peaks_3 = find_peaks(hist_3, height=0.3, width=2)

x = np.arange(0, 256)

peak_locations_1 = [x[i] for i in peaks_1[0]]
peak_locations_2 = [x[i] for i in peaks_2[0]]
peak_locations_3 = [x[i] for i in peaks_3[0]]

fig, axs = plt.subplots(3, 1, figsize=(12, 10))

axs[0].bar(np.arange(256), hist, color='gray')
axs[0].set_xlim([0, 256])

axs[0].scatter(x = peak_locations_1, y = hist[peak_locations_1], color = 'r')
axs[0].set_title("Textureless")

axs[1].bar(np.arange(256), hist_texture, color='gray')
axs[1].set_xlim([0, 256])
axs[1].scatter(x = peak_locations_2, y = hist_texture[peak_locations_2], color = 'r')
axs[1].set_title("Texture")

axs[2].bar(np.arange(256), hist_3, color='gray')
axs[2].set_xlim([0, 256])
axs[2].scatter(x = peak_locations_3, y = hist_3[peak_locations_3], color = 'r')
axs[2].set_title("Intermediate")

plt.tight_layout()
plt.show()


## **Blob Detection**

In [ ]:
blob = blob_detection(empty_image)

fig, axs = plt.subplots(1, 2, figsize=(10, 5))

axs[0].imshow(blob)
axs[0].set_title("Blob")

axs[1].imshow(empty_image)
axs[1].set_title("Image")
plt.show()

## **Line Detection**

In [ ]:
line_image, linecenters, U, V = detect_and_draw_lines(empty_image, 100, 200)
print(f"Number of Lines: {len(linecenters)}")

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(10, 5))

axs[0].imshow(line_image)
axs[0].set_title("Blob")

axs[1].imshow(empty_image)
axs[1].set_title("Image")

axs[2].quiver(linecenters[:, 0], linecenters[:, 1], U, V, angles='xy', scale_units='xy', scale=1)
axs[2].set_title("Center Distribution")
plt.show()

In [ ]:
## lines in an image with texture
line_texture_image, linecenters, U, V = detect_and_draw_lines(color_image_1, 100, 200)
print(f"Number of Lines: {len(linecenters)}")
fig, axs = plt.subplots(1, 3, figsize=(10, 5))

axs[0].imshow(line_texture_image)
axs[0].set_title("Blob")

axs[1].imshow(color_image_1)
axs[1].set_title("Image")

axs[2].quiver(linecenters[:, 0], linecenters[:, 1], U, V, angles='xy', scale_units='xy', scale=1)
axs[2].set_title("Center Distribution")
plt.show()

In [ ]:
## lines in an image with texture
line_texture_image, linecenters, U, V = detect_and_draw_lines(color_image_2, 100, 200)
print(f"Number of Lines: {len(linecenters)}")
fig, axs = plt.subplots(1, 3, figsize=(10, 5))

axs[0].imshow(line_texture_image)
axs[0].set_title("Blob")

axs[1].imshow(color_image_2)
axs[1].set_title("Image")

axs[2].quiver(linecenters[:, 0], linecenters[:, 1], U, V, angles='xy', scale_units='xy', scale=1)
axs[2].set_title("Center Distribution")
plt.show()